# Plot times series based the gridded fields 

In [38]:
using Dates
using DelimitedFiles 
using NCDatasets
using PyPlot
using DIVAnd
const plt=PyPlot
using PyCall
cmocean = PyCall.pyimport("cmocean")
include("../src/InterpCalanus.jl")
include("../src/config.jl")

"#456A30"

## Files and directories

In [42]:
isdir(resdir) ? @debug("already there") : mkpath(resdir)
isdir(figdir) ? @debug("already there") : mkpath(figdir)
isdir(resfigdir) ? @debug("already there") : mkpath(resfigdir)
isfile(datafile) ? @debug("already downloaded") : download(datafileURL, datafile)

## Reading monthly and yearly results

In [32]:
resfile_yearmonth_fidmarchicus = joinpath(resdir, "calanus_finmarchicus_year_month.nc")
resfile_yearmonth_helgolandicus = joinpath(resdir, "calanus_helgolandicus_year_month.nc")
resfile_year_fidmarchicus = joinpath(resdir, "calanus_finmarchicus_year.nc")
resfile_year_helgolandicus = joinpath(resdir, "calanus_helgolandicus_year.nc")

isfile(resfile_yearmonth_fidmarchicus) & isfile(resfile_yearmonth_helgolandicus) & 
isfile(resfile_year_fidmarchicus) & isfile(resfile_year_helgolandicus)

true

In [33]:
lonyear_fid, lat_year_fid, times_year, field_year_fid, error_year = InterpCalanus.read_results(resfile_year_fidmarchicus)
_, _, _, field_year_helgo, _ = InterpCalanus.read_results(resfile_year_helgolandicus);
_, _, times_month, field_yearmonth_fid, error_month = InterpCalanus.read_results(resfile_yearmonth_fidmarchicus);
_, _, _, field_yearmonth_helgo, _ = InterpCalanus.read_results(resfile_yearmonth_helgolandicus);

In [34]:
yearlist = Dates.year.(times_year);

### Compute mean for the time series

In [35]:
fmean1 = InterpCalanus.compute_time_mean(field_year_fid)
fmean2 = InterpCalanus.compute_time_mean(field_year_helgo)
fmean3 = InterpCalanus.compute_time_mean(field_yearmonth_fid)
fmean4 = InterpCalanus.compute_time_mean(field_yearmonth_helgo);

## Make plots
### Years

In [55]:
textdict = Dict("facecolor"=>"white", "alpha"=>0.85, "edgecolor"=>".85")

Dict{String, Any} with 3 entries:
  "facecolor" => "white"
  "edgecolor" => ".85"
  "alpha"     => 0.85

In [58]:
fig = plt.figure()
ax = plt.subplot(111)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
ax.plot(times_year, fmean1, "ko-", markersize=2, label="Calanus finmarchicus", zorder=3)
ax.plot(times_year, fmean2, "o-", linewidth=2, color=mycolor, markersize=2, label="Calanus helgolandicus", zorder=3)
ax.plot(times_year[argmax(fmean1)], maximum(fmean1), "ko", ms=7, zorder=5)
ax.plot(times_year[argmax(fmean2)], maximum(fmean2), "o", color=mycolor, ms=7, zorder=5)


text1 = ax.text(Dates.Year(1) + times_year[argmax(fmean1)], maximum(fmean1), ha="left", va="bottom", 
    " $(round(maximum(fmean1), digits=1))\n($(yearlist[argmax(fmean1)]))",
    zorder=4)
text1.set_bbox(textdict)

text2 = ax.text(Dates.Year(1) + times_year[argmax(fmean2)], maximum(fmean2), ha="left", va="bottom", 
    " $(round(maximum(fmean2), digits=1))\n($(yearlist[argmax(fmean2)]))", color=mycolor,
    zorder=4)
text2.set_bbox(textdict)


ax.text(times_year[end], fmean1[end], " Calanus finmarchicus", va="center")
ax.text(times_year[end], fmean2[end], " Calanus helgolandicus", color=mycolor, va="center")
ax.set_ylabel("Mean\nabundance", rotation=0, ha="right", fontsize=18)
plt.savefig(joinpath(resfigdir, "time_series_year.png"))
plt.close()

### Months

In [62]:
fig = plt.figure()
ax = plt.subplot(111)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
ax.plot(times_month, fmean3, "ko-", markersize=2, label="Calanus fidmarchicus")
ax.plot(times_month, fmean4, "o-", markersize=2, label="Calanus helgolandicus", color=mycolor)
plt.legend()
ax.set_ylabel("Mean\nabundance", rotation=0, ha="right", fontsize=18)
plt.savefig(joinpath(resfigdir, "time_series_month"))
plt.close()

## Error field

In [64]:
mean_error_year = InterpCalanus.compute_time_mean(error_year)
mean_error_month = InterpCalanus.compute_time_mean(error_month);

In [66]:
fig = plt.figure()
ax = plt.subplot(111)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
ax.plot(times_year, mean_error_year, "ko-", markersize=2)
ax.set_ylabel("Mean\nerror", rotation=0, ha="right", fontsize=18)
plt.savefig(joinpath(resfigdir, "time_series_error_year"))
plt.close()